In [24]:
import pandas as pd
import numpy as np
import random
import tensorflow.keras as keras
import matplotlib.pyplot as plt

# pandas is useful for data manipulation and analysis
# we use the read_csv function inside the pandas library to read the dataset downloaded
# we have to specify the file path
train = pd.read_csv('Data_Set/sign_mnist_train.csv')
test = pd.read_csv('Data_Set/sign_mnist_test.csv')

# we use the array function inside the numpy library to convert the data set to numpy arrays
train_data = np.array(train, dtype = 'float32')
test_data = np.array(test, dtype = 'float32')

In [ ]:
# Plot a few images and label
# The main purpose of this block of code is to check whether our data has been properly read and is working

# List of alphabets according to the numbers assigned inside the labels, 0 for A, 1 for B and so on
class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']

# this is to generate a number between 1 and the total number of rows that exist in the training set
# basically load an image data from all the images available in csv format 
i = random.randint(1, train.shape[0])
fig1, ax1 = plt.subplots(figsize = (1.5, 1.5))

# [i, 1:] is used to take the entire pixel array after the first column, as first column is for label
# we use reshape (28, 28) to display it in the form of 28 * 28, to view the image
plt.imshow(train_data[i, 1:].reshape((28, 28)), cmap = 'gray')
# priniting the alphabet for which the sign stands for train_data[i, 0] gives us the numerical value stored in that row
print("Label for the image/sign is:", class_names[int(train_data[i, 0])])

In [ ]:
# Data distribution and visualization
# This is a good method to check whether the data set is balanced or not

fig = plt.figure(figsize = (18, 18))
ax1 = fig.add_subplot(221)
train['label'].value_counts().plot(kind = 'bar', ax = ax1)
ax1.set_ylabel('Count')
ax1.set_title('Label')